<a href="https://colab.research.google.com/github/rubyvanrooyen/observation_planning/blob/main/MeerKAT_spectral_and_spatial_resolution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.constants as phys

from IPython.display import display
from ipywidgets import interact, widgets

## Spectral resolution

MeerKAT spectral line modes:
* **32k wide band** has a bandwidth of 856 MHz and 32k channels
* **32k narrow band** has a bandwidth of 107 MHz and 32k channels

Zoom modes with much narrower bandwidth have had some engineering tests, but remain untested for science.

In [ ]:
dtype=[('Name', 'S15'),
       ('CentreFreq', float),  # MHz
       ('Bandwidth', float),  # MHz
       ('Channels', int),
       ('ChannelWidth', float),  # kHz
      ]
MKT_modes = np.array([('c856M32k', 1284.000 , 856.000, 32768, 26.123),
                      ('c856M32k_n107M', 1400.000, 107.000, 32768, 3.265)],
                    dtype=dtype)

Spectral resolution for 32k modes = $\Delta v$ the velocity span (km/s):    
$\Delta v \approx \Delta f \frac{c}{f}$    
with $\Delta f$ the channel width, $f$ the centre frequency and $c$ the speed of light in a vacuum.

In [ ]:
c = phys.c/1e3  # km/s
for MKT_mode in MKT_modes:
    df = MKT_mode['ChannelWidth'] * 1e3
    f =  MKT_mode['CentreFreq'] * 1e6
    dv = df/f*c  # km/s
    print('Bandwidth {} MHz over {} channels for channel width {} km/s at {} GHz'
          .format(MKT_mode['Bandwidth'],
                  MKT_mode['Channels'],
                  dv,
                  MKT_mode['CentreFreq']/1e3))

Bandwidth 856.0 MHz over 32768 channels for channel width 6.099282227674455 km/s at 1.284 GHz
Bandwidth 107.0 MHz over 32768 channels for channel width 0.69915883955 km/s at 1.4 GHz


OH maser frequencies of interest, are the strong 1665 MHz and weaker 1667 MHz lines, as well as the 1612 MHz satellite line.

Assuming the narrowband mode allow for the observation centre frequency to be selected, setting $f_c\approx 1640\,$MHz will allow simultaneous observations of all 3 lines.

The MeerKAT passband needs to be taken into account as well, since the 1667 MHz line is very close to the MeerKAT L-band (900 MHz - 1670 MHz) upper frequency.

In [ ]:
center = (1667.+1612.)/2.
start = center - (107./2.)
end = center + (107./2.)

print('Frequency range covered during narrow band observation [{}, {}] MHz @ fc={} MHz center frequence'
       .format(start, end, center))

end = 1670
center = end - (107./2.)
start = center - (107./2.)

print('Proposed settings fc={} MHz for frequency range [{}, {}] MHz'
       .format(center, start, end))

Frequency range covered during narrow band observation [1586.0, 1693.0] MHz @ fc=1639.5 MHz center frequence
Proposed settings fc=1616.5 MHz for frequency range [1563.0, 1670] MHz


## Spatial resolution

In [ ]:
# basic constants
degrad=180./np.pi
arcminute_deg=60.
arcsecond_deg=3600.
rad_arcmin = degrad * arcminute_deg
rad_arcsec = degrad * arcsecond_deg

D = 13.5  # m -- dish diameter
Bmax = 7.7  # km

### Field of view
FWHM of primary beam

In [ ]:
# observation centre frequency
label = widgets.Text(value='Centre Frequency ', disabled=True, layout={'width':'130px'})
cen_freq = widgets.Text(value='1284', placeholder='MHz', disabled=False, layout={'width':'55px'})
unit = widgets.Text(value='[MHz]', disabled=True, layout={'width':'55px'})
ui = widgets.HBox([label, cen_freq, unit])
display(ui)

In [ ]:
f = float(cen_freq.value)*1e6  # Hz
wavelength = phys.c/f  # m

In [ ]:
Dl = D / wavelength  # diameter in wavelengths
FWHMpb = 1.02 * (rad_arcmin/Dl)  # [arcmin]
print('MeerKAT FOV {:.3} [arcmin] (generally assumed to be around 1 deg)'.format(FWHMpb))  # FOV of the interferometer

MeerKAT FOV 60.6 [arcmin] (generally assumed to be around 1 deg)


### Resolution
FWHM of synthesized beam

In [ ]:
B = Bmax*1e3  # m
Bl = B / wavelength
FWHMsb = 1.02 * (rad_arcsec/Bl)  # [arcmin]
print('MeerKAT Resolution @ {} [MHz] = {:.3} [arcsec]'
      .format(f/1e6, FWHMsb))  # equivalent to the interferometer resolution

MeerKAT Resolution @ 1284.0 [MHz] = 6.38 [arcsec]


Apparent size = 3438 [arcmin] * $\frac{\mbox{Diameter [km]}}{\mbox{Distance [km]}}$

In [ ]:
Callisto_diameter = 2.* 2500.  # km
Jupiter_diameter = 2. * 70000.   # km
Earth_distance = 630e6  # km

Callisto_size = rad_arcsec * (Callisto_diameter/Earth_distance)  # [arcsec]
Jupiter_size = rad_arcsec * (Jupiter_diameter/Earth_distance)  # [arcsec]
print('Jupiter angular size approx {:.3} [arcsec]'.format(Jupiter_size))
print('Callisto angular size approx {:.3} [arcsec]'.format(Callisto_size))

Jupiter angular size approx 45.8 [arcsec]
Callisto angular size approx 1.64 [arcsec]
